In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import glob
import json


def get_folders():
    pattern = 'dataset/*/out/meta_llama-3.1-405b-instruct_augmented_prompt/agent_protocol.json'
    folders = [folder.split('/')[1] for folder in glob.glob(pattern)]
    return folders

# if __name__ == "__main__":
folders = get_folders()
print(json.dumps(folders, indent=2))
with open('done.json', 'w') as f:
    json.dump(folders, f, indent=2)

In [21]:
import os
from pathlib import Path


def generate_github_actions_workflow(dockerfiles: list[Path], language_model_id: str):
    if len(dockerfiles) == 0 or not dockerfiles:
        return None
    dockerfiles = list(set(dockerfiles))
    workflow = {
        'name': 'Build and Publish Docker Images',
        'on': {'push': {'paths': [f'dataset/*/out/reproduction/{language_model_id}'], 'branches': ['**']}},
        'jobs': {
            'build-and-push': {
                'runs-on': 'ubuntu-latest',
                'steps': [
                    {'name': 'Checkout code', 'uses': 'actions/checkout@v4', "with": {"sparse-checkout": "dataset"}},
                    {'name': 'Set up Docker Buildx', 'uses': 'docker/setup-buildx-action@v2'},
                    {
                        'name': 'Login to GitHub Container Registry',
                        'uses': 'docker/login-action@v1',
                        'with': {
                            'registry': 'ghcr.io',
                            'username': '${{ github.actor }}',
                            'password': '${{ secrets.GITHUB_TOKEN }}'
                        }
                    }
                ]
            }
        }
    }

    for dockerfile in dockerfiles:
        commit_hash = dockerfile.parent.parent.parent.parent.name
        image_name = f"ghcr.io/${{{{ github.repository_owner }}}}/{language_model_id}-{commit_hash}-reproduction"
        
        workflow['jobs']['build-and-push']['steps'].append({
            'name': f'Build and push {commit_hash}',
            'uses': 'docker/build-push-action@v2',
            'with': {
                'context': str(dockerfile.parent.relative_to(Path(os.path.abspath("")).parent.as_posix())),
                'file': str(dockerfile.relative_to(Path(os.path.abspath("")).parent.as_posix())),
                'push': True,
                'tags': image_name
            }
        })

    return workflow

In [29]:
import glob
import os
from pathlib import Path
import json
import shutil
import subprocess
from collections import defaultdict

import yaml

from masterthesis.agent.GitAgent import GitAgent

# Global variables
data_path = Path(os.path.abspath("")).parent / "dataset"
print("data_path", data_path)
dockerfile_paths_by_language_model = defaultdict(list)

lm_mapping = {
        "gpt-4o-mini": "gpt-4o-mini",
        "open-mistral-nemo": "mistral-nemo",
        "gemini-1.5-pro-001": "gemini-1.5-pro",
        "gemini-1.5-flash-001": "gemini-1.5-flash",
        "claude-3-5-sonnet@20240620": "claude-3.5-sonnet",
        "claude-3-haiku@20240307": "claude-3-haiku"
    }


def get_language_model(file_path):
    lm = file_path.split("/")[-2].split("_")[0]
    
    return lm

def process_files():
    glob_path = data_path / "*/out/*/chat_log.jsonl"
    path_list = glob.glob(glob_path.as_posix())
    print([path for path in path_list])

    paths_prep = [Path(path).parts[-2] for path in path_list]
    paths_prep.sort()
    print(paths_prep)


    data = {lm: {'Attempts': 0, 'Compilation Success': 0, 'Test Suite Success': 0, 'Errors': 0} for lm in set(paths_prep)}
    error_set = {lm: set() for lm in set(paths_prep)}

    for path in path_list:
        
        path = Path(path)
        language_model_id = path.parts[-2]
        commit_hash = path.parts[-4]
        # print(language_model_id, commit_hash, path)

        lm = language_model_id
        data[lm]['Attempts'] += 1
        
        solution_path = path.parent / "solution.json"
        error_path = path.parent / "error.txt"
        
        
        if solution_path.exists():
            with open(solution_path, "r") as f:
                solution = json.load(f)
            if solution["compilation_has_succeeded"] and not solution["test_has_succeeded"]:
                data[lm]['Compilation Success'] += 1
                # process_successful_compilation(commit_hash, language_model_id)
            if solution["test_has_succeeded"] and solution["compilation_has_succeeded"]:
                data[lm]['Test Suite Success'] += 1
                process_successful_compilation(commit_hash, language_model_id)
        
        if error_path.exists():
            with open(error_path, "r") as f:
                file_text = f.read()
                data[lm]['Errors'] += 1
                error_set[lm].add(file_text)
        
        if not solution_path.exists() and not error_path.exists():
            data[lm]['Errors'] += 1

    return data, error_set

def process_successful_compilation(commit_hash, language_model_id):
    commit_data_path = Path(os.path.abspath("")).parent / "dataset" / commit_hash
    reproduction_path = commit_data_path / "out" / "reproduction" / language_model_id
    os.makedirs(reproduction_path, exist_ok=True)

    repo_path = commit_data_path / "repo"
    repo_slug = commit_data_path / "repo_slug.txt"

    with open(repo_slug, "r") as f:
        repo_slug = f.read().strip()
        project = repo_slug.split("/")[1]

    final_state = commit_data_path / "out"/language_model_id / "final_state.diff"
    if not final_state.exists():
        solution_path = commit_data_path / "out" / language_model_id / "solution.json"
        with open(solution_path, "r") as f:
            solution = json.load(f)
        updated_files = solution.get("updated_files", {})
        paths_to_copy = updated_files.keys()
        for path in paths_to_copy:
            if not path:
                continue
            os.makedirs(reproduction_path / Path(path).parent, exist_ok=True)
            with open(reproduction_path/path, "w") as f:
                f.write(updated_files[path])
        return paths_to_copy
    with open(final_state, "r") as f:
        patch = f.read()
    if not patch or not isinstance(patch, str):
        print("No valid patch")
        shutil.rmtree(reproduction_path)
        return
    git_agent = GitAgent(repo_path=commit_data_path/"repo",commit_hash=commit_hash,github_slug=repo_slug)
    # try:
    
    git_agent.discard_changes()
    print("Applying", commit_data_path/"repo", language_model_id)
    try:
        paths_to_copy = git_agent.apply_diff(patch)

        for path in paths_to_copy:
            if not path:
                continue
            os.makedirs(reproduction_path / Path(path).parent, exist_ok=True)
            shutil.copyfile(commit_data_path/"repo"/path, reproduction_path/path)
            if not (reproduction_path/path).exists():
                print("File not copied", path)
                shutil.rmtree(reproduction_path)
                raise Exception("File not copied")
        git_agent.discard_changes()

        copy_instructions = ["COPY " + str(path) + " /" + project + "/" + str(path) for path in paths_to_copy]
        copy_instructions = list(set(copy_instructions))
        copy_instructions.sort()

        with open(reproduction_path / "patch.txt", "w") as f:
            f.write(patch)

        

        docker_file = f"""FROM ghcr.io/chains-project/breaking-updates:{commit_hash}-breaking

{"\n".join(copy_instructions)}
    """

        dockerfile_path = reproduction_path / "Dockerfile"
        dockerfile_paths_by_language_model[language_model_id].append(dockerfile_path)
        with open(dockerfile_path, "w") as f:
            f.write(docker_file)


        real_language_model_id = language_model_id.split("_")[0]
        simplified_lm_id = lm_mapping.get(real_language_model_id, real_language_model_id)

        registry_url = "ghcr.io/lukvonstrom/masterthesis-implementation"

        replicate_script = f"""#!/bin/bash

docker pull ghcr.io/chains-project/breaking-updates:{commit_hash}-breaking
docker build -t {simplified_lm_id}-{commit_hash}-reproduction {reproduction_path}
docker run ghcr.io/chains-project/breaking-updates:{commit_hash}-breaking > pre.txt 2>&1
docker run {simplified_lm_id}-{commit_hash}-reproduction > post.txt 2>&1

# Tag and push the reproduction image to GitHub package registry
docker tag {simplified_lm_id}-{commit_hash}-reproduction {registry_url}:{simplified_lm_id}-{commit_hash}-reproduction
docker push {registry_url}:{simplified_lm_id}-{commit_hash}-reproduction


docker save -o /root/docker-images/{simplified_lm_id}-{commit_hash}-reproduction.tar {simplified_lm_id}-{commit_hash}-reproduction
gzip /root/docker-images/{simplified_lm_id}-{commit_hash}-reproduction.tar"""
        with open(reproduction_path / "replicate.sh", "w") as f:
            f.write(replicate_script)
        os.chmod(reproduction_path / "replicate.sh", 0o755)
    except Exception as e:
        print("Diff application failed", e)
        # shutil.rmtree(reproduction_path)


    

    
# except Exception as e:
#     print("Normal Diff application failed", e)
#     # shutil.rmtree(reproduction_path)

    

# finally:
    git_agent.discard_changes()



repro_data_path = Path(os.path.abspath("")).parent / "dataset" / "*" / "out" / "reproduction" / "*"
repro_paths = glob.glob(repro_data_path.as_posix())
for reproduction_path in repro_paths:
    if not Path(reproduction_path).exists():
        continue
    # shutil.rmtree(reproduction_path)

# Main execution
data, errors = process_files()
print(data)
print(errors)




for language_model_id, dockerfile_paths in dockerfile_paths_by_language_model.items():
    workflow = generate_github_actions_workflow(dockerfile_paths, language_model_id)
    if workflow:
        with open(f"../.github/workflows/build-images-{language_model_id}.yml", "w") as f:
            yaml.dump(workflow, f, sort_keys=False)

data_path /root/thesis/masterthesis-implementation-gpt/dataset
['/root/thesis/masterthesis-implementation-gpt/dataset/489aad6060454d0b7b34a144e0b345c5a3a199f5/out/claude-3-haiku@20240307_augmented_prompt/chat_log.jsonl', '/root/thesis/masterthesis-implementation-gpt/dataset/489aad6060454d0b7b34a144e0b345c5a3a199f5/out/gpt-4o-mini_augmented_prompt/chat_log.jsonl', '/root/thesis/masterthesis-implementation-gpt/dataset/489aad6060454d0b7b34a144e0b345c5a3a199f5/out/meta_llama-3.1-70b-instruct_augmented_prompt/chat_log.jsonl', '/root/thesis/masterthesis-implementation-gpt/dataset/489aad6060454d0b7b34a144e0b345c5a3a199f5/out/claude-3-5-sonnet@20240620_augmented_prompt/chat_log.jsonl', '/root/thesis/masterthesis-implementation-gpt/dataset/489aad6060454d0b7b34a144e0b345c5a3a199f5/out/open-mistral-nemo_augmented_prompt/chat_log.jsonl', '/root/thesis/masterthesis-implementation-gpt/dataset/489aad6060454d0b7b34a144e0b345c5a3a199f5/out/gemini-1.5-pro-001_augmented_prompt/chat_log.jsonl', '/root/the

In [27]:
import os
import glob
from pathlib import Path

success_marker = "[INFO] BUILD SUCCESS"
failure_marker = "[INFO] BUILD FAILURE"
data_path = Path(os.path.abspath("")).parent / "dataset"
failure_to_success = 0

def check_status_transition(pre_path):
    parent = Path(pre_path).parent
    language_model_id = parent.parts[-1]
    commit = parent.parts[-4]
    post_path = parent / "post.txt"
    
    pre_content = ""
    post_content = ""
    try:
        with open(pre_path, "r") as f:
            pre_content = f.read()
        with open(post_path, "r") as f:
            post_content = f.read()
    except Exception as e:
        print(f"Error reading files: {pre_path} {post_path}")
    
    pre_status = "Failure" if failure_marker in pre_content else "Success" if success_marker in pre_content else "Unknown"
    post_status = "Success" if success_marker in post_content else "Failure" if failure_marker in post_content else "Unknown"
    
    if pre_status == "Failure" and post_status == "Success":
        # print(f"Transitioned from Failure to Success: {pre_path} {language_model_id} {commit}")
        global failure_to_success
        failure_to_success += 1
    elif pre_status != post_status:            
        print(f"Status changed from {pre_status} to {post_status}: {pre_path}")
        print(f"  Language Model: {language_model_id}")
        print(f"  Commit: {commit}")
    else:
        print(f"Status unchanged: {pre_path}")

# Get all pre.txt files
path_list = glob.glob(os.path.join(data_path, "*/out/*/*/pre.txt"))
replicate_files = glob.glob(str(data_path / '**' / 'out' / 'reproduction' / '**' / 'replicate.sh'), recursive=True)

# Check status transition for each path
for path in path_list:
    check_status_transition(path)


print(f"Success to Failure: {failure_to_success} / {len(replicate_files)}")

Success to Failure: 74 / 74
